In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_2_Choosing_Investment_Strategies/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-18 13:49:12.705868


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

This problem can be formulated as a mixed-integer linear programming (MILP) problem with chance constraints. Here are the parameters, decision variables, objective function, and constraints for this problem:

**Parameters:**

- \(A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]\): Costs of investing in each strategy
- \(R \in \mathbb{R}^{n \times 10}\): Matrix of historical returns for each strategy, where \(n\) is the number of past investment outcomes
- \(B = [1000, 1100, 900, 1200]\): Vector of possible budget scenarios
- \(p = [0.55, 0.4, 0.04, 0.01]\): Vector of probabilities for each budget scenario
- \(r = 2\): Risk parameter for the mean-variance model

**Decision Variables:**

- \(x \in \{0, 1\}^{10}\): Binary decision variables indicating whether to invest in a strategy or not
- \(z \geq 0\): Auxiliary variable representing the expected value of the investment
- \(s \geq 0\): Auxiliary variable representing the standard deviation (risk) of the investment

**Objective Function:**

The objective is to maximize the expected return on investment, while minimizing the risk (variance):

\[\max (z - r \cdot s)\]

**Constraints:**

1. Budget constraint: The total investment cost should not exceed the budget in any scenario with a probability higher than 0.4. This can be represented by a chance constraint:

\[P\left(\sum_{i=1}^{10} A_i \cdot x_i \leq B_k\right) \geq 1 - 0.4, \quad k = 1, 2, 3, 4\]

This constraint can be transformed into a deterministic constraint using the concept of quantile functions:

\[\sum_{i=1}^{10} A_i \cdot x_i \leq \text{Quantile}[0.6, B \cdot p^T]\]

2. Mean-variance constraints: These constraints ensure that the auxiliary variables \(z\) and \(s\) accurately represent the expected return and standard deviation of the investment, respectively.

\[z = \frac{1}{n} \sum_{j=1}^{n} \sum_{i=1}^{10} R_{ji} \cdot x_i\]

\[s^2 = \frac{1}{n} \sum_{j=1}^{n} \left(\sum_{i=1}^{10} R_{ji} \cdot x_i - z\right)^2\]

Note that the second constraint is nonlinear due to the square term. To linearize this constraint, we can introduce another auxiliary variable \(w \geq 0\) and add the following constraints:

\[w \geq \left(\sum_{i=1}^{10} R_{ji} \cdot x_i - z\right)^2, \quad j = 1, \ldots, n\]

\[s^2 = \frac{1}{n} \sum_{j=1}^{n} w_j\]

This formulation allows us to transform the nonlinear constraint into a linear one, making the optimization problem a MILP. This problem can be solved using off-the-shelf optimization software to find the optimal investment strategy under the uncertain budget scenario.

In [6]:
print(response_1_text)

This problem can be formulated as a mixed-integer linear programming (MILP) problem with chance constraints. Here are the parameters, decision variables, objective function, and constraints for this problem:

**Parameters:**

- \(A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]\): Costs of investing in each strategy
- \(R \in \mathbb{R}^{n \times 10}\): Matrix of historical returns for each strategy, where \(n\) is the number of past investment outcomes
- \(B = [1000, 1100, 900, 1200]\): Vector of possible budget scenarios
- \(p = [0.55, 0.4, 0.04, 0.01]\): Vector of probabilities for each budget scenario
- \(r = 2\): Risk parameter for the mean-variance model

**Decision Variables:**

- \(x \in \{0, 1\}^{10}\): Binary decision variables indicating whether to invest in a strategy or not
- \(z \geq 0\): Auxiliary variable representing the expected value of the investment
- \(s \geq 0\): Auxiliary variable representing the standard deviation (risk) of the investment

**Objective Funct

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

To solve this problem using Pyomo, we first need to install the necessary packages. You can install Pyomo and its dependencies using the following command:

```bash
pip install pyomo
```

Now, let's write the Pyomo code to solve the MILP problem. I will use sample data for the historical returns matrix R, as it was not provided in the problem statement.

```python
import numpy as np
import pyomo.environ as pyo

# Sample data for historical returns matrix R
np.random.seed(123)
n = 100  # number of past investment outcomes
R = np.random.rand(n, 10)

# Parameters
A = np.array([80, 340, 410, 50, 180, 221, 15, 348, 191, 225])
B = np.array([1000, 1100, 900, 1200])
p = np.array([0.55, 0.4, 0.04, 0.01])
r = 2

# Define the Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(10), within=pyo.Binary)
model.z = pyo.Var(within=pyo.NonNegativeReals)
model.s = pyo.Var(within=pyo.NonNegativeReals)

# Objective function
model.obj = pyo.Objective(expr=model.z - r * model.s, sense=pyo.maximize)

# Budget constraint
budget_constraint = sum(A[i] * model.x[i] for i in range(10)) <= np.quantile(B @ p, 0.6)
model.budget_constraint = pyo.Constraint(expr=budget_constraint)

# Mean-variance constraints
n_inv = np.linalg.inv(np.identity(n) - (1/n) * np.ones((n, n)))
Z = R @ model.x.vec
model.mean_constraint = pyo.Constraint(expr=model.z == (1/n) * sum(Z[j] for j in range(n)))

model.aux_vars = pyo.Var(range(n), within=pyo.NonNegativeReals)
model.aux_vars_constraints = pyo.ConstraintList()
for j in range(n):
    model.aux_vars_constraints.add(model.aux_vars[j] >= (Z[j] - model.z) ** 2)

model.std_constraint = pyo.Constraint(expr=model.s ** 2 == (1/n) * sum(model.aux_vars[j] for j in range(n)))

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.obj))
print("Investment strategy:", [pyo.value(model.x[i]) for i in range(10)])
print("Expected return:", pyo.value(model.z))
print("Standard deviation:", pyo.value(model.s))
```

This code defines the Pyomo model, adds the decision variables, objective function, and constraints. It then solves the model using the GLPK solver and prints the results. Replace the sample data for R with your actual data to get the solution for your specific problem.

In [10]:
print(response_2_text)

To solve this problem using Pyomo, we first need to install the necessary packages. You can install Pyomo and its dependencies using the following command:

```bash
pip install pyomo
```

Now, let's write the Pyomo code to solve the MILP problem. I will use sample data for the historical returns matrix R, as it was not provided in the problem statement.

```python
import numpy as np
import pyomo.environ as pyo

# Sample data for historical returns matrix R
np.random.seed(123)
n = 100  # number of past investment outcomes
R = np.random.rand(n, 10)

# Parameters
A = np.array([80, 340, 410, 50, 180, 221, 15, 348, 191, 225])
B = np.array([1000, 1100, 900, 1200])
p = np.array([0.55, 0.4, 0.04, 0.01])
r = 2

# Define the Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(10), within=pyo.Binary)
model.z = pyo.Var(within=pyo.NonNegativeReals)
model.s = pyo.Var(within=pyo.NonNegativeReals)

# Objective function
model.obj = pyo.Objective(expr=model.z - r * mode

### Code Executability

In [11]:
import numpy as np
import pyomo.environ as pyo

# Sample data for historical returns matrix R
np.random.seed(123)
n = 100  # number of past investment outcomes
R = np.random.rand(n, 10)

# Parameters
A = np.array([80, 340, 410, 50, 180, 221, 15, 348, 191, 225])
B = np.array([1000, 1100, 900, 1200])
p = np.array([0.55, 0.4, 0.04, 0.01])
r = 2

# Define the Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(10), within=pyo.Binary)
model.z = pyo.Var(within=pyo.NonNegativeReals)
model.s = pyo.Var(within=pyo.NonNegativeReals)

# Objective function
model.obj = pyo.Objective(expr=model.z - r * model.s, sense=pyo.maximize)

# Budget constraint
budget_constraint = sum(A[i] * model.x[i] for i in range(10)) <= np.quantile(B @ p, 0.6)
model.budget_constraint = pyo.Constraint(expr=budget_constraint)

# Mean-variance constraints
n_inv = np.linalg.inv(np.identity(n) - (1/n) * np.ones((n, n)))
Z = R @ model.x.vec
model.mean_constraint = pyo.Constraint(expr=model.z == (1/n) * sum(Z[j] for j in range(n)))

model.aux_vars = pyo.Var(range(n), within=pyo.NonNegativeReals)
model.aux_vars_constraints = pyo.ConstraintList()
for j in range(n):
    model.aux_vars_constraints.add(model.aux_vars[j] >= (Z[j] - model.z) ** 2)

model.std_constraint = pyo.Constraint(expr=model.s ** 2 == (1/n) * sum(model.aux_vars[j] for j in range(n)))

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.obj))
print("Investment strategy:", [pyo.value(model.x[i]) for i in range(10)])
print("Expected return:", pyo.value(model.z))
print("Standard deviation:", pyo.value(model.s))

AttributeError: 'IndexedVar' object has no attribute 'vec'

### Solution Correctness